# Toolkits
工具箱是工具的集合，这些工具被设计成一起用于特定的任务。他们有方便的加载方法。有关可用的现成工具包的完整列表，请访问[Integrations](https://python.langchain.com/docs/integrations/toolkits/)

所有工具包都公开了一个`get_tools`法，该方法返回一个工具列表。因此你可以

In [43]:
from dotenv import load_dotenv,find_dotenv

# 加载.env文件中的环境变量
load_dotenv(find_dotenv())

ImportError: cannot import name 'load_dotenv' from 'dotenv' (/slurm/home/yrd/shaolab/daiyizheng/.conda/envs/langechain/lib/python3.10/site-packages/dotenv/__init__.py)

```python
# Initialize a toolkit
toolkit = ExampleTookit(...)

# Get list of tools
tools = toolkit.get_tools()

# Create agent
agent = create_agent_method(llm, tools, prompt)
```

##  工具
工具是代理、链或 LLM 用来与世界交互的接口。它们结合了以下几点：
- 工具名称
- 该工具是什么的描述
- 该工具输入的 JSON 模式
- 要调用的函数
- 工具的结果是否应直接返回给用户

拥有所有这些信息非常有用，因为这些信息可用于构建采取行动的系统！名称、描述和 JSON 模式可用于提示 LLM，以便它知道如何指定要执行的操作，然后调用的函数相当于执行该操作。

工具的输入越简单，LLMs就越容易使用它。许多代理只能使用具有单个字符串输入的工具。有关代理类型列表以及哪些代理类型可处理更复杂的输入，请参阅此文档

重要的是，名称、描述和 JSON 模式（如果使用）都用于提示中。因此，它们清晰并准确描述如何使用该工具非常重要。如果 LLM 不了解如何使用该工具，您可能需要更改默认名称、描述或 JSON 模式。

## 默认
让我们看看如何使用工具。为此，我们将使用内置工具。



In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
tool.name

'wikipedia'

In [4]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [5]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [6]:
tool.run({"query": "langchain"})

ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=langchain&format=json&action=query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x2af8401fab00>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

### 自定义默认工具
我们还可以修改参数的内置名称、描述和JSON模式。

在定义参数的JSON模式时，重要的是输入与函数保持相同，因此不应该更改这一点。但是您可以轻松地为每个输入定义自定义描述。

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

In [ ]:
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

## 定义自定义工具
在构造您自己的代理时，您需要为它提供一个可以使用的工具列表。除了实际调用的函数外，该工具还由几个组件组成
- `name` (str)是必需的，并且在提供给代理的一组工具中必须是唯一的
- `description` (str)是可选的，但建议使用，因为代理使用它来确定工具的使用情况
- `args_schema`  (Pydantic BaseModel)，可选但推荐使用，可用于提供更多信息(例如，少量示例)或对预期参数进行验证。

定义工具有多种方法。在本指南中，我们将介绍如何实现两个函数
1. 一个生成的搜索函数，它总是返回字符串“LangChain”。
2. 一个将两个数相乘的乘数函数

这里最大的区别是第一个函数只需要一个输入，而第二个函数需要多个输入。许多代理只处理需要单个输入的函数，因此了解如何处理这些函数非常重要。在大多数情况下，定义这些自定义工具是相同的，但存在一些差异。





In [8]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

### @tool decorator

这个@tool装饰器是定义自定义工具的最简单方法。装饰器默认使用函数名作为工具名，但可以通过传递一个字符串作为第一个参数来覆盖这一点。另外，装饰器将使用函数的文档字符串作为工具的描述——所以必须提供一个文档字符串。

In [9]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [10]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [11]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [12]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


您还可以通过将工具名称和JSON参数传递给工具装饰器来定制工具名称和JSON参数。

In [13]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
search-tool(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}
True


### 子类BaseTool
您还可以通过继承BaseTool类来显式地定义自定义工具。这提供了对工具定义的最大控制，但是工作要多一些。

In [14]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return "LangChain"

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

In [15]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

custom_search
useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


In [16]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

Calculator
useful for when you need to answer questions about math
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
True


### StructuredTool dataclass
你也可以使用StructuredTool数据类。这种方法是前两种方法的混合。它比从BaseTool类继承更方便，但提供了比仅仅使用装饰器更多的功能。

In [17]:
def search_function(query: str):
    return "LangChain"


search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

In [18]:
print(search.name)
print(search.description)
print(search.args)

Search
Search(query: str) - useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


您还可以定义自定义args模式来提供有关输入的更多信息。

In [19]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

您还可以定义自定义`args`模式来提供有关输入的更多信息

In [20]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

In [22]:
print(calculator.name)
print(calculator.description)
print(calculator.args)

Calculator
Calculator(a: int, b: int) -> int - multiply numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


### Handling Tool Errors
当工具遇到错误且未捕获异常时，代理将停止执行。如果希望代理继续执行，可以引发`ToolException`并相应地设置`handle tool error`。\

抛出`ToolException`时，代理不会停止工作，而是根据工具的`handle tool error`变量处理异常，并将处理结果作为观察返回给代理，并以红色打印。

您可以将`handle_tool_error`设置为True，将其设置为一个统一的字符串值，或将其设置为一个函数。如果它被设置为一个函数，那么这个函数应该接受一个`ToolException`作为参数并返回一个str值。

请注意，只引发`ToolException`是无效的。工具的`handle_tool_error`默认值为`False`，需要先设置该工具的`handle_tool_error`。

In [23]:
from langchain_core.tools import ToolException


def search_tool1(s: str):
    raise ToolException("The search tool1 is not available.")

In [24]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
)

search.run("test")

ToolException: The search tool1 is not available.

现在，让我们将`handle_tool_error`设置为`True`

In [25]:
search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=True,
)

search.run("test")

'The search tool1 is not available.'

我们还可以定义一种自定义方法来处理工具错误

In [26]:
def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )


search = StructuredTool.from_function(
    func=search_tool1,
    name="Search_tool1",
    description="A bad tool",
    handle_tool_error=_handle_error,
)

search.run("test")

'The following errors occurred during tool execution:The search tool1 is not available.Please try another tool.'

## 工具作为OpenAI函数
本笔记本介绍了如何使用LangChain工具作为OpenAI功能。

In [27]:
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

In [45]:
model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key="sk-mgI3GClWked2tJAODf82F69f0a78485f87BfC5Ac2f331b1a")

In [46]:
tools = [MoveFileTool()]
functions = [convert_to_openai_function(t) for t in tools]

In [47]:
functions[0]

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'type': 'object',
  'properties': {'source_path': {'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [48]:
message = model.invoke(
    [HumanMessage(content="move file foo to bar")], functions=functions
)

In [ ]:
message

In [ ]:
message.additional_kwargs["function_call"]

在OpenAI聊天模型中，我们还可以用`bind_functions`自动绑定和转换类函数对象

In [ ]:
model_with_functions = model.bind_functions(tools)
model_with_functions.invoke([HumanMessage(content="move file foo to bar")])

或者我们可以使用更新的OpenAI API，使用`tools `和`tool_choice `，而不是使用通过`ChatOpenAI.bind_tools`的`functions`和`function_call`

In [ ]:
model_with_tools = model.bind_tools(tools)
model_with_tools.invoke([HumanMessage(content="move file foo to bar")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_btkY3xV71cEVAOHnNa5qwo44', 'function': {'arguments': '{\n  "source_path": "foo",\n  "destination_path": "bar"\n}', 'name': 'move_file'}, 'type': 'function'}]})